# Gym Progress Analysis Project

This project processes and visualizes gym workout data exported from two tracking apps.
It includes data cleaning, standardization, and creation of derived metrics like *work rate*, *frequency*, and *max_1RM*.

In [17]:
import pandas as pd

#2 main graphs i want to make: 
#work rate and frequency (muscle group) of over time.
#1 rpm max (specific lift) over time.
df1 = pd.read_csv('data_2022-2023.csv')
df2 = pd.read_csv("data_2024-2025.csv")


In [18]:
#Code for cleaning dataframes
def clean_workout_data(df):
    #Standardize headers
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(' ', '_')
    )

    #Drop unnecessary columns safely
    df = df.drop(columns=[
        'workout_name', 'distance', 'seconds', 'rpe', 'notes',
        'kcal', 'name', 'bodyweight', 'set_order', 'category'
    ], errors='ignore')

    #Rename inconsistent columns (if different app formats)
    rename_map = {
        'exercise': 'exercise_name',
    }
    df = df.rename(columns=rename_map)

    #Clean and standardize date
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.date

    #Remove invalid reps (≤ 0 or > 100)
    if 'reps' in df.columns:
        df = df[(df['reps'].fillna(0) > 0) & (df['reps'] <= 40)].reset_index(drop=True)

    #Remove invalid weight (≤ 0)
    if 'weight' in df.columns:
        #Convert to float to avoid dtype assignment warnings
        df['weight'] = pd.to_numeric(df['weight'], errors='coerce').astype(float)
        
        #Fix very large weights (accidental extra zero)
        df.loc[df['weight'] > 1000, 'weight'] = df['weight'] / 10
        
        #Remove zero or negative weights
        df = df[df['weight'].fillna(0) > 0].reset_index(drop=True)

    #Drop rows with critical missing data
    critical_cols = [col for col in ['date', 'exercise_name', 'workout_end'] if col in df.columns]
    df = df.dropna(subset=critical_cols).reset_index(drop=True)

    #Add week and year columns if date is valid
    if 'date' in df.columns:
        df['week'] = df['date'].apply(lambda x: x.isocalendar().week if pd.notnull(x) else None)
        df['year'] = df['date'].apply(lambda x: x.isocalendar().year if pd.notnull(x) else None)

    return df


#Clean both app dataframes
df1 = clean_workout_data(df1)
df2 = clean_workout_data(df2)

In [19]:
#I noticed that df1 has the date split into start and end. 
#for simplicity, I will just use the same format with df2 and have a single date column.
#I will first find the duration of work out by subtracting start from end, then convert to minutes.
df1['workout_start'] = pd.to_datetime(df1['workout_start'])
df1['workout_end'] = pd.to_datetime(df1['workout_end'])

#Calculate duration in minutes
df1['duration_min'] = (df1['workout_end'] - df1['workout_start']).dt.total_seconds() / 60

#One problem with this dataset is that i often input the end date to be 1 day before the start date by mistake.
#To fix this, I will add the day difference if the duration is negative.
df1['duration_min'] = df1['duration_min'].apply(lambda x: x + 24*60 if x < 0 else x)

#Keep just the date for mergin purposes later
df1['date'] = df1['workout_start'].dt.date

#Before cleaning df1, date was not in cols so week and year cols were not created.
#Now, I will create them here.
df1['year'] = pd.to_datetime(df1['date']).dt.isocalendar().year
df1['week'] = pd.to_datetime(df1['date']).dt.isocalendar().week

#Drop the start and end columns if no longer needed
df1 = df1.drop(columns=['workout_start', 'workout_end'])


In [20]:
#Convert duration like "1h 30min" or "45min" into total minutes
df2['duration_min'] = (
    df2['duration']
    .str.replace(" ", "") 
    .apply(lambda x: pd.to_timedelta(x).total_seconds() / 60 if pd.notna(x) else None)
    .astype('float')
)

#Drop original duration columns safely
df1 = df1.drop(columns=['duration'], errors='ignore')
df2 = df2.drop(columns=['duration'], errors='ignore')


In [21]:

#For some of the data, i often forgot to stop the workout, causing inflated values for duration.
#To fix this, I replace any duration values over 180 minutes with the means of other durations that are less than 180 minutes.
def fix_inflated_durations(df, col='duration_min', max_valid=180):
    #Ensure column is float type to avoid dtype assignment warnings
    df[col] = df[col].astype(float)

    #Compute mean of valid durations
    valid_mean = df.loc[df[col] <= max_valid, col].mean()

    #Replace inflated values
    df.loc[df[col] > max_valid, col] = valid_mean

    #Round to whole minutes
    df[col] = df[col].round(0).astype(int)

    return df


# Apply to both dataframes
df1 = fix_inflated_durations(df1)
df2 = fix_inflated_durations(df2)

In [22]:
#Check the name variations in both datasets and see how to map them
print(df1['exercise_name'].dropna().unique())
print(df2['exercise_name'].dropna().unique())

['Pull Up' 'T-Bar row' 'Bird dog row' 'Dumbbell delt '
 'Dumbbell Bicep Curl' 'Hammer Curl' 'Bench Press' 'Dips'
 'Single Leg KB Overhead Press' 'Dumbbell Lateral Raises' 'Squat'
 'Incline Dumbbell Press' 'Banded pull apart' 'Medicine ball slam'
 'Crunches' 'Hammer strenght high row' 'Lat pushdown' 'Cable Row'
 'Shoulder Dumbbell Press' 'Cable delt ' 'Heavy lateral raise + 10 shrug'
 'Lat pulldown' 'Hammer strenght medium row' 'Pushdowns' 'Leg Extensions'
 'Calf Raises' 'Preacher curl' 'High to low cable chest fly'
 'Low to high cable chest fly' 'Leg Curls' 'Dumbbell RDL'
 'Over-head tricep cable' 'Grip strenghten exercise' 'Single arm pushdown'
 'Incline dumbbell curl' 'Lateral cable raise' 'Abs roller']
['Incline Bench Press (Smith Machine)' 'Lat Pulldown (Cable)'
 'Bench Press (Smith Machine)' 'Preacher Curl (Machine)'
 'Triceps Extension' 'Hack Squat' 'Lying Leg Curl (Machine)'
 'Leg Extension (Machine)' 'Lateral Raise (Cable)' 'Reverse Fly (Cable)'
 'Calf Press on Leg Press' 'Seat

In [23]:
#Normalize exercise names to lowercase and normalize machine type
def normalize_exercise_names(df):
    df['exercise_name'] = (
        df['exercise_name']
        .str.lower()
        .str.strip()
        .str.replace(r'\s+', ' ', regex=True)
        .str.strip()
    )
    return df


df1 = normalize_exercise_names(df1)

df2 = normalize_exercise_names(df2)


#remove exercises that do not target specific muscle groups (like warmup exercises and cardio)
#For the first dataset, it was early in my trainning journey so i did more questionable exercises.
#I will remove any it as it does not contributed to my current goals.
unwanted_exercises = ['stretching', 'running (treadmill)', 'banded pull apart',
                      'bird dog row', 'single leg kb overhead press',
                      'medicine ball slam', 'grip strenghten exercise',
                      "lower back hyper extend", 'box single leg jumb',
                      'static squad jumb', "piston squad", "jm press", 
                      'decline bench press (barbell)']
df1 = df1[~df1['exercise_name'].isin(unwanted_exercises)]
df2 = df2[~df2['exercise_name'].isin(unwanted_exercises)]

#Now I translate both datasets to have consistent exercise names
#Also group up similar exercises into one standard name
name_map = {
    #chest
    'bench press': 'bench press (barbell)',
    'incline dumbbell press': 'incline bench press (dumbbell)',
    'push up plyo explosive': 'push up',
    'high to low cable chest fly': 'cable crossover',
    'low to high cable chest fly': 'cable crossover',

    #triceps
    'dips': 'triceps dip',
    'over-head tricep cable': 'triceps extension',
    'single arm pushdown': 'triceps extension',
    'pushdowns': 'triceps extension',

    #biceps
    'dumbbell bicep curl': 'bicep curl (cable)',
    'hammer curl': 'bicep curl (cable)',
    'incline dumbbell curl': 'bicep curl (cable)',
    'preacher curl': 'preacher curl (machine)',

    #shoulders
    'shoulder dumbbell press': 'overhead press (smith machine)',
    'lateral cable raise': 'lateral raise (cable)',
    'dumbbell lateral raises': 'lateral raise (dumbbell)',
    'heavy lateral raise + 10 shrug': 'lateral raise (dumbbell)',
    'cable delt': 'reverse fly (cable)',
    'dumbbell delt': 'reverse fly (cable)',

    #back
    'scalp pull up': 'pull up',
    'lat pulldown': 'lat pulldown (cable)',
    'lat pulldown - wide grip (cable)': 'lat pulldown (cable)',
    'lat pushdown': 'lat pulldown (cable)',
    'hammer strenght high row': 'seated wide-grip row (cable)',
    't-bar row': 'seated wide-grip row (cable)',
    't bar row': 'seated wide-grip row (cable)',
    'hammer strenght medium row': 'seated row (machine)',
    'cable row': 'seated row (cable)',

    #legs
    'leg press': 'squat (barbell)',
    'squat': 'squat (barbell)',
    'hack squat': 'hack squat (machine)',
    'pendulum squat': 'pendulum squat (machine)',
    'leg extensions': 'leg extension (machine)',
    'leg curls': 'lying leg curl (machine)',
    'dumbbell rdl': 'seated leg curl (machine)',
    'romanian deadlift': 'seated leg curl (machine)',
    'rdl': 'seated leg curl (machine)',
    'seated leg curl': 'seated leg curl (machine)',
    'romanian deadlift (barbell)': 'seated leg curl (machine)',

    #calves
    'calf raises': 'seated calf raise (machine)',
    'calf press on leg press': 'calf press (machine)',
    'calf press on seated leg press': 'calf press (machine)',

    #core
    'crunches': 'crunch (machine)',
    'abs roller': 'crunch (machine)',
    'leg raises': 'leg raises'
}


#Apply mappings to both dataframes
df1["exercise_name"] = df1["exercise_name"].replace(name_map)
df2["exercise_name"] = df2["exercise_name"].replace(name_map)


print(df1['exercise_name'].dropna().unique())
print(df2['exercise_name'].dropna().unique())

['pull up' 'seated wide-grip row (cable)' 'reverse fly (cable)'
 'bicep curl (cable)' 'bench press (barbell)' 'triceps dip'
 'lateral raise (dumbbell)' 'squat (barbell)'
 'incline bench press (dumbbell)' 'crunch (machine)'
 'lat pulldown (cable)' 'seated row (cable)'
 'overhead press (smith machine)' 'seated row (machine)'
 'triceps extension' 'leg extension (machine)'
 'seated calf raise (machine)' 'preacher curl (machine)' 'cable crossover'
 'lying leg curl (machine)' 'seated leg curl (machine)'
 'lateral raise (cable)']
['incline bench press (smith machine)' 'lat pulldown (cable)'
 'bench press (smith machine)' 'preacher curl (machine)'
 'triceps extension' 'hack squat (machine)' 'lying leg curl (machine)'
 'leg extension (machine)' 'lateral raise (cable)' 'reverse fly (cable)'
 'calf press (machine)' 'seated row (cable)' 'bicep curl (cable)'
 'seated wide-grip row (cable)' 'reverse fly (machine)' 'chest fly'
 'seated leg curl (machine)' 'pendulum squat (machine)'
 'squat (smith mac

In [24]:
#After making the columns consistent between both datasets, I can now combine them for easier analysis later
df_combined = pd.concat([df1, df2], ignore_index=True)

In [25]:
#Mapping muscle groups to exercises
exercise_to_group = {
    # Chest
    'bench press (barbell)': 'chest',
    'bench press (smith machine)': 'chest',
    'incline bench press (dumbbell)': 'chest',
    'incline bench press (smith machine)': 'chest',
    'decline bench press (barbell)': 'chest',
    'chest press (machine)': 'chest',
    'chest fly': 'chest',
    'cable crossover': 'chest',
    'push up': 'chest',

    # Triceps
    'triceps extension': 'triceps',
    'triceps dip': 'triceps',
    'jm press': 'triceps',

    # Shoulders
    'overhead press (smith machine)': 'shoulders',
    'shoulder press (machine)': 'shoulders',
    'lateral raise (dumbbell)': 'shoulders',
    'lateral raise (cable)': 'shoulders',
    'reverse fly (cable)': 'shoulders',
    'reverse fly (machine)': 'shoulders',
    'shrug (smith machine)': 'shoulders',
    'shrug (dumbbell)': 'shoulders',

    # Back
    'lat pulldown (cable)': 'back',
    'seated row (cable)': 'back',
    'seated row (machine)': 'back',
    'seated wide-grip row (cable)': 'back',
    't bar row': 'back',
    'pull up': 'back',
    'reverse fly (cable)': 'back',
    'reverse fly (machine)': 'back',

    # Biceps
    'bicep curl (cable)': 'biceps',
    'preacher curl (machine)': 'biceps',

    # Legs
    'squat (barbell)': 'legs',
    'squat (smith machine)': 'legs',
    'hack squat (machine)': 'legs',
    'pendulum squat (machine)': 'legs',
    'leg press': 'legs',
    'leg extension (machine)': 'legs',
    'lying leg curl (machine)': 'legs',
    'seated leg curl (machine)': 'legs',
    'seated calf raise (machine)': 'legs',
    'calf press (machine)': 'legs',

    # Core
    'crunch (machine)': 'core',
    'leg raises': 'core'
}


df_combined['muscle_group'] = df_combined['exercise_name'].replace(exercise_to_group).fillna('other')


#Check which exercises were mapped to 'other' muscle group
print(df_combined.loc[df_combined['muscle_group'] == 'other', 'exercise_name'].unique())

[]


In [26]:
#Now i will start working on calculating the work rate of a certain muscle group over time
#Step 1: Finding out the total volume of a certain muscle group per session
df_combined['volume'] = df_combined['weight'] * df_combined['reps']

#Step 2: Count sets per muscle group per day
df_combined['muscle_sets'] = df_combined.groupby(['date', 'muscle_group'])['muscle_group'].transform('count')

#Step 3: Count total sets per day
df_combined['total_sets_day'] = df_combined.groupby('date')['date'].transform('count')

#Step 4: Proportional duration per muscle group
df_combined['duration_muscle_group'] = (
    df_combined['duration_min'] * (df_combined['muscle_sets'] / df_combined['total_sets_day'])
)

#Step 5: Compute total volume per day per muscle group
df_combined['total_volume_muscle_group'] = (
    df_combined.groupby(['date', 'muscle_group'])['volume'].transform('sum')
)

#Step 6: Compute one work rate per date + muscle_group
df_combined['work_rate'] = (
    df_combined['total_volume_muscle_group'] / df_combined['duration_muscle_group']
)

In [27]:
#For frequency per week, I group the date, muscle_group, year, week to find unique days trained per muscle group per week. 
#Drop duplicates per muscle group per day
temp = df_combined[['date', 'muscle_group', 'year', 'week']].drop_duplicates()

#Count how many unique days each muscle group appears in that week
freq = (
    temp.groupby(['year', 'week', 'muscle_group'])['date']
        .transform('count')
)

#Map that frequency back to df2
df_combined = df_combined.merge(
    temp.assign(frequency=freq),
    on=['date', 'muscle_group', 'year', 'week'],
    how='left'
)



In [28]:
#For the last graph, i will find the max 1RM for a specific lift over time of that day by using Brzycki formula from Matt Brzycki
#1RM = weight / (1.0278 - 0.0278 * reps. I choose this formula because it is more accurate for 5-10 reps range, which is my trainning style
df_combined['1RM'] = df_combined['weight'] / (1.0278 - 0.0278 * df_combined['reps'])

#funtion above will output the 1RM for each set, now i will find the max 1RM for a specific lift over time
#Group by date and exercise, take the max 1RM
max_1rm = df_combined.groupby(['date', 'exercise_name'])['1RM'].transform('max')
df_combined['max_1RM'] = max_1rm


In [29]:
#Reorder columns
desired_order = [
    'date', 'year', 'week', 
    'muscle_group', 'exercise_name', 
    'weight', 'reps', 'work_rate', 
    'volume', 'frequency', 'max_1RM'
]

#Reorder columns — keep your chosen first, then any leftover columns
df_combined = df_combined[[col for col in desired_order if col in df_combined.columns]]

#Save cleaned combined data to new CSV
df_combined.to_csv('cleaned_gym_data.csv', index=False)

After finalizing the cleaned and structured dataset in Python, I exported it as a CSV and imported it into **Power BI** to visualize my workout trends, performance progression, and overall consistency.

### Data Preparation in Power BI

Before building the visuals, I did some final data transformations directly in Power BI:
- **Removed the “year” column** since Power BI automatically recognizes and creates a built-in **date hierarchy** (Year → Quarter → Month → Day) from the date column.  
- **Kept the “week” column** as a useful metric to represent ongoing training progression over time. 
- Created a few **custom DAX calculations**:
  - To compute the **true 1RM (one-rep max)** for each exercise across each week.
  - To calculate **target values** representing strength goals (e.g., target 1RM progression).


These steps ensured the data behaved dynamically when filtering by exercise, muscle group, or time.


### Dashboard Design

The dashboard consists of **three main visuals**, all designed to show different aspects of my training journey:

1. **Work Rate and Frequency Combined Chart**  
   - A dual-axis chart combining **bar (work rate)** and **line (frequency)** to show how often I trained a muscle group and the effort intensity across time.  
   - Both share the same legend — **muscle group** — allowing me to compare consistency and workload side by side.  
   - This visualization helped me see how consistent training patterns directly influence performance metrics.

2. **Weeks Trained Over Time** 
   - Decided to add this on to see how many weeks i have trained or have recoreded in a year
   - A line chart displaying the number of **weeks trained per year**, providing a clear overview of my long-term consistency.  
   - It visually highlights gaps in my training data and periods of high or low commitment (e.g., during school terms or breaks).

3. **1RM Progression Chart (Custom DAX)**    
   - The DAX formula aggregates the **max 1RM** for each exercise, showing clear peaks and plateaus over time.  

Bellow is an interactive dashboard


### Power BI Dashboard Overview
!["Dashboard"](https://github.com/TonyDeDev/personal-trainning-data-analysis/blob/75438f232efe55eb0f1e3c321cb71b4b27249ed3/dashboard/dashboardOverView.png)

### Reflections

After transforming the data and creating visualizations in Power BI, several key trends became clear.

At First, there is a correlation between training **frequency** and **1 RM**. Weeks where I trained a muscle group more consistently tend to align with periods of noticeable strength gains within 2023-2025. The **work rate** also trails closely behind this progression pattern, suggesting that both the quantity and intensity of training contribute to performance growth over same time period.

When analyzing specific muscle groups—particularly the **bench press (barbell)**, **squat (barbell)**, and **lat pulldown (cable)**—these trends become even clearer. I consider these three lifts as representative of my upper body, lower body, and back strength progression, respectively.

For **bench press (barbell)**, consistent increase in frequency and work rate throughout the year directly correlated with higher 1RM estimates and faster strength progression beside 2022.

!["Dashboard - Bench Press (Barbell)"](https://github.com/TonyDeDev/personal-trainning-data-analysis/blob/75438f232efe55eb0f1e3c321cb71b4b27249ed3/dashboard/dashboardBenchPress.png)

For **squat (barbell)**, training frequency shows a stronger correlation with lift progression, while work rate does not display a clear relationship with performance improvements.

!["Dashboard - Squat (Barbell)"](https://github.com/TonyDeDev/personal-trainning-data-analysis/blob/75438f232efe55eb0f1e3c321cb71b4b27249ed3/dashboard/dashboardSquat.png")

For **lat pulldown (cable)**, In 2022, training frequency was relatively high, but performance gains were minimal. After 2023, frequency dropped and gradually increased again, showing a stronger correlation with overall strength progression during this phase.

!["Dashboard - Lat Pulldown (Cable)"](https://github.com/TonyDeDev/personal-trainning-data-analysis/blob/75438f232efe55eb0f1e3c321cb71b4b27249ed3/dashboard/dashboardPullDown.png)

### Conclusion

- In 2022, there was little to no correlation between frequency (reflecting my split type) and work rate (intensity). This likely reflects the early stage of my training journey, where I did not follow a structured split, and despite lifting with some intensity, the quality of my form reduced the effectiveness of those sessions.

- From 2023 onward, a stronger correlation emerged between frequency, work rate, and strength progression. However, gaps remain due to limited data recording—particularly in 2023 and 2024, when I switched to training at home and missed some sessions.

- Despite these limitations, the trends provide a clear sense of direction: continuing the current approach, maintaining or increasing intensity, and gradually improving frequency should support continued strength gains and progression in key lifts.